In [34]:
import os
os.chdir("prepared_corpora/msrp_matching/")


In [126]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division

import itertools
import codecs
from os import path
import csv



In [86]:
def load_MSRP(filename):
    with codecs.open(filename,'r', b"utf-8" ) as fh:
        nlines = 0
        for line in fh.readlines():
            nlines+=1
            if nlines==1:
                continue
            isparaphrase, id1, id2, str1, str2 = line.split("\t") #the quality fielld is 1 for phraphrases and 0 for not
            yield(int(isparaphrase)==1, (int(id1),str1.strip()),(int(id2),str2.strip()))

            
            
                

In [62]:
def create_eval_corpora(base_paraphrases, folder, max_corruption_level=10):
    """We want to create the baseline corpus as a new line seperated sentences, so that it works well with Sorchers URAE system.
    Thus all the metadata is stored in seperate files referencing the line numbers"""
    
    global phrase_line_num 
    phrase_line_num = 0 #line numebrs are always refered to after icrementing them
    openned_filehandles = []
    try:
        phrases_fh = codecs.open(path.join(folder, "phrases.txt"),'w', b"utf-8" )
        openned_filehandles.append(phrases_fh)

        microsoft_ids_fh = open(path.join(folder, "microsoft_ids.txt"),'w')
        openned_filehandles.append(microsoft_ids_fh)
        microsoft_ids_csv = csv.writer(microsoft_ids_fh)
        microsoft_ids_csv.writerow(["phrase_line_number","microsoft_id"])

        paraphrases_fh = open(path.join(folder, "paraphrases.txt"),"w")
        openned_filehandles.append(paraphrases_fh)
        paraphrases_csv = csv.writer(paraphrases_fh)
        paraphrases_csv.writerow(["phrase_line_num", "paraphrase_line_num"])

        ##Recorder Functions
        def add_phrase(phrase):
            global phrase_line_num
            phrases_fh.write(phrase)
            phrases_fh.write("\n")
            phrase_line_num+=1
            return phrase_line_num
        
   
        for (isparaphrase,(m_id1,phrase1),(m_id2,phrase2)) in base_paraphrases:
            #Add the phrases, and the corruptions
            ln1 = add_phrase(phrase1)
            ln2 = add_phrase(phrase2)
            #add to the record of microsoft ids
            microsoft_ids_csv.writerow([ln1,m_id1])
            microsoft_ids_csv.writerow([ln2,m_id2])

            if (isparaphrase):
                #add the paraphases, in both directions
                paraphrases_csv.writerow([ln1,ln2])
                paraphrases_csv.writerow([ln2,ln1])
            
    finally:
        for fh in openned_filehandles:
            fh.close()

In [89]:
corp_gen = itertools.chain(
        load_MSRP("../../base_corpora/MSRP/msr_paraphrase_test.txt"),
        load_MSRP("../../base_corpora/MSRP/msr_paraphrase_train.txt")
)

create_eval_corpora(corp_gen,".")

####Here one must go and generate the Parsed versions (and while at it make the embeddings)

In [90]:
import nltk
import numpy as np


In [91]:
corpus_reader = nltk.corpus.BracketParseCorpusReader(".",["parsed.txt"])
parse_trees = np.array(corpus_reader.parsed_sents(), dtype=object).squeeze();

In [92]:
def labels_only(tree):
    if isinstance(tree, nltk.Tree):
        labels = [tree.label()]
        
        for child in tree:
            child_label = labels_only(child)
            if child_label:
                labels.append(child_label)
        return tuple(labels)
            
    else:
        return None
    


In [93]:
import collections

count = collections.Counter([labels_only(tree) for id,tree in enumerate(parse_trees)])
collections.Counter(count.values())
#~700 syntactic matches idk how many are already paraphrases.

Counter({1: 10212, 2: 564, 3: 56, 4: 8, 7: 2, 11: 2, 6: 1, 8: 1, 12: 1})

In [94]:
structures = collections.defaultdict(set)

for id,tree in enumerate(parse_trees,1):
    struct = labels_only(tree) 
    structures[struct].add(id)
    

In [181]:
para_ids = np.loadtxt("./paraphrases.txt", delimiter=",",skiprows=1, dtype=np.int)
para_id_map = {row[0]:row[1] for row in para_ids}

structural_matches = []
for struct_set in structures.values():
    if len(struct_set)>1:
        for id in struct_set:
            for other_id in struct_set.difference({id}):
                if not (id in para_id_map) or para_id_map[id]!=other_id:
                    leaves1 = set(parse_trees[id-1].leaves())
                    leaves2 = parse_trees[other_id-1].leaves()
                    if len(leaves1.intersection(leaves2))/len(leaves1) < 0.70: # must be 50% difference in bag of words 
                        structural_matches.append((id,other_id))
    


In [182]:
len(structural_matches)

32

In [163]:
structural_matches

[(2546, 6130), (6130, 2546)]

In [164]:
parse_trees[2546-1].leaves()

[u'Trish',
 u'York',
 u',',
 u'a',
 u'spokeswoman',
 u'for',
 u'Boeing',
 u',',
 u'declined',
 u'to',
 u'comment',
 u'on',
 u'the',
 u'deal',
 u'.']

In [165]:
parse_trees[6130-1].leaves()

[u'Anthony',
 u'Citrano',
 u',',
 u'a',
 u'representative',
 u'for',
 u'WhenU',
 u',',
 u'declined',
 u'to',
 u'comment',
 u'on',
 u'the',
 u'case',
 u'.']